# ITU1770-4 Loudness

## Algorithms to measure audio programme loudness and true-peak audio level
Recommendation ITU-R BS.1770-4 (10/2015, Electronic Publication, Geneva, 2017)

see also:

**Loudness normalisation and permitted maximum level of audio signals**
EBU R 128-2014, June 2014

**Loudness Metering: EBU Mode metering to supplement Loudness normalisation in accordance with EBU R 128**
R 128 LOUDNESS NORMALIZATION, TECH 3341 (January 2016)
 
**Note:**
No measurement gate is applied in the code below, yet!!! See `get_LUFS_Stereo` in util.py for a version that handles mono audio, this needs re-checked for stereo files.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import soundfile as sf
import util

In [ ]:
fs = 44100 

def get_dBTruePeak(x):  # x.shape = (channels; samples)
    tmp = signal.resample_poly(x/4, up=32*1, down=1*1, axis=-1, window=('kaiser', 5.0))
    tmp = tmp[:, int(50/1000*(44100*32))::]  # avoid evaluation of first samples
    # where filter needs to settle up
    L = 20*np.log10(4*np.max(np.abs(tmp), axis=-1))
    return L

def get_LUFS_Stereo(x, fs):  # x.shape = (channels 1,2 or 3; samples)
    if fs == 48000:
        # Pre-Filter @ 48kHz
        bPRE = (1.53512485958697, -2.69169618940638, 1.19839281085285)
        aPRE = (1, -1.69065929318241, 0.73248077421585)
        # RLB-Filter @ 48kHz
        bRLB = (1, -2, 1)
        aRLB = (1, -1.99004745483398, 0.99007225036621)
    elif fs == 44100:  # redesigned by fs446: analog filter fit of the 48 kHz
        # filter, then bilinear transform to 44.1kHz
        # Pre-Filter @ 44.1kHz
        bPRE = (1.53089320672149, -2.65101332422074, 1.16905574510282)
        aPRE = (1, -1.66364666025175, 0.712582287855323)
        # RLB-Filter @ 44.1kHz
        bRLB = (1, -2, 1)
        aRLB = (1, -1.98917270016531, 0.989202007770735)
    else:
        bPRE = (1,0,0)
        aPRE = (1,0,0)
        bRLB = (1,0,0)
        aRLB = (1,0,0)
        print('! unknown fs, filter thru !')
        
    sos = np.array([bPRE, aPRE, bRLB, aRLB])
    sos = np.reshape(sos, (2,6))
    if False:
        [f, H] = signal.sosfreqz(sos[0,:], worN=2**12, whole=False, fs=fs)
        plt.semilogx(f,20*np.log10(np.abs(H)), lw=4, label='PRE')
        [f, H] = signal.sosfreqz(sos[1,:], worN=2**12, whole=False, fs=fs)
        plt.semilogx(f,20*np.log10(np.abs(H)), lw=4, label='RLB')
        [f, H] = signal.sosfreqz(sos[:], worN=2**12, whole=False, fs=fs)
        plt.semilogx(f,20*np.log10(np.abs(H)), 'k', label='PRE+RLB')
        plt.grid(True)
        plt.xlabel('f / Hz')
        plt.ylabel('A / dB')
        plt.legend()    
    y = signal.sosfilt(sos, x, axis=-1)
    L = -0.691 + 10*np.log10(np.sum(np.var(y, axis=-1)))
    return L

In [ ]:
x = np.array([10**(-23/20) * np.sin( 2*np.pi*1000/fs * np.arange(0, 1*fs) ),
              10**(-23/20) * np.sin( 2*np.pi*1000/fs * np.arange(0, 1*fs) )])
print(x.shape)
print('ITU 1770-4 loudness = ', get_LUFS_Stereo(x, fs), 'LUFS')
print('ITU 1770-4 true peak = ', get_dBTruePeak(x), 'dBTP')

In [ ]:
# stereo file handling
data, samplerate = sf.read('../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli/pnoise_ref.wav')
x = np.array(data.T)
print(x.shape)
print('ITU 1770-4 loudness = ', get_LUFS_Stereo(x, samplerate), 'LUFS')  # transpose data!
print('ITU 1770-4 true peak = ', np.max(get_dBTruePeak(x)), 'dBTP')

# mono file handling
data, samplerate = sf.read('../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli/pnoise_ref_mono.wav')
x = np.array([data.T])
print(x.shape)
print('ITU 1770-4 loudness no Gate= ', get_LUFS_Stereo(x, samplerate), 'LUFS')  # transpose data!
print('ITU 1770-4 loudness with Gate= ', util.get_LUFS_Stereo(x.T, samplerate, mono_gated=True), 'LUFS')  # transpose data!
print('ITU 1770-4 true peak = ', np.max(get_dBTruePeak(x)), 'dBTP')